In [1]:
%load_ext autoreload
%autoreload 2

import os
os.environ['KMP_WARNINGS'] = 'off'
import sys
import git

import uproot as ut
import awkward as ak
import numpy as np
import math
import vector
import sympy as sp

import re
from tqdm import tqdm
import timeit
import re

sys.path.append( git.Repo('.', search_parent_directories=True).working_tree_dir )
from utils import *

In [2]:
from functools import partial

In [3]:
module = fc.eightb.feynnet

In [4]:
files = module.feynnet_signal_list

In [5]:
f_signal = ObjIter(files)

In [6]:
open_tree = partial(Tree, altfile='test_{base}', report=False)

In [7]:
# signal = f_signal.apply(open_tree, report=True)
# del signal

In [16]:
signal = f_signal.apply(open_tree, parallel=True, report=True)

100%|██████████| 33/33 [01:53<00:00,  3.44s/it]


In [56]:
%%time 
nthreads = 3
with mp.pool.ThreadPool(nthreads) as pool:
    result = pool.map(open_tree, f_signal[:5])

CPU times: user 12.3 s, sys: 639 ms, total: 12.9 s
Wall time: 13.4 s


In [57]:
# %%time 
nthreads = 1
with mp.pool.Pool(nthreads) as pool:
    result = pool.map(open_tree, f_signal[:5])

MaybeEncodingError: Error sending result: '[<utils.classUtils.Tree.Tree object at 0x7f8b926199d0>, <utils.classUtils.Tree.Tree object at 0x7f8b5125ceb0>]'. Reason: 'TypeError("cannot pickle '_global_parameters' object")'

In [58]:
result

In [27]:

signal = f_signal.apply( open_tree, parallel=True, report=True, nthreads=20 )

 42%|████▏     | 14/33 [01:52<02:32,  8.01s/it]


Process ForkPoolWorker-23:
Process ForkPoolWorker-24:
Process ForkPoolWorker-27:
Process ForkPoolWorker-22:
Process ForkPoolWorker-26:
Process ForkPoolWorker-21:
Process ForkPoolWorker-28:
Process ForkPoolWorker-25:
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
  File "/uscms_data/d3/ekoenig/mambaforge/envs/py-env/lib/python3.9/site-packages/multiprocess/process.py", line 315, in _bootstrap
    self.run()
  File "/uscms_data/d3/ekoenig/mambaforge/envs/py-env/lib/python3.9/site-packages/multiprocess/process.py", line 315, in _bootstrap
    self.run()
Traceback (most recent call last):
Traceback (most recent call last):
  File "/uscms_data/d3/ekoenig/mambaforge/envs/py-env/lib/python3.9/site-packages/multiprocess/process.py", line 315, in _bootstrap
    self.run()
  File "/uscms_data/d3/ekoenig/mambaforge/envs/py-env/lib/python3.9/site-packages/multiprocess/process.py", line 108, in run
    self._target(*self._args, **self._kwarg

In [25]:
f_signal.apply( print, parallel=True) 

/store/user/ekoenig/8BAnalysis/NTuples/2018/feynnet/NMSSM_XYY_YToHH_8b/NMSSM_XYY_YToHH_8b_MX_600_MY_250/ntuple.root
/store/user/ekoenig/8BAnalysis/NTuples/2018/feynnet/NMSSM_XYY_YToHH_8b/NMSSM_XYY_YToHH_8b_MX_600_MY_300/ntuple.root
/store/user/ekoenig/8BAnalysis/NTuples/2018/feynnet/NMSSM_XYY_YToHH_8b/NMSSM_XYY_YToHH_8b_MX_700_MY_250/ntuple.root
/store/user/ekoenig/8BAnalysis/NTuples/2018/feynnet/NMSSM_XYY_YToHH_8b/NMSSM_XYY_YToHH_8b_MX_700_MY_300/ntuple.root
/store/user/ekoenig/8BAnalysis/NTuples/2018/feynnet/NMSSM_XYY_YToHH_8b/NMSSM_XYY_YToHH_8b_MX_700_MY_350/ntuple.root
/store/user/ekoenig/8BAnalysis/NTuples/2018/feynnet/NMSSM_XYY_YToHH_8b/NMSSM_XYY_YToHH_8b_MX_800_MY_250/ntuple.root
/store/user/ekoenig/8BAnalysis/NTuples/2018/feynnet/NMSSM_XYY_YToHH_8b/NMSSM_XYY_YToHH_8b_MX_800_MY_300/ntuple.root
/store/user/ekoenig/8BAnalysis/NTuples/2018/feynnet/NMSSM_XYY_YToHH_8b/NMSSM_XYY_YToHH_8b_MX_800_MY_350/ntuple.root
/store/user/ekoenig/8BAnalysis/NTuples/2018/feynnet/NMSSM_XYY_YToHH_8b/N

100%|██████████| 33/33 [00:00<00:00, 129357.04it/s]


[None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None]

In [22]:
pool = mp.Pool(processes=8)

In [24]:
mp.pool.ThreadPool

multiprocess.pool.ThreadPool

In [21]:
signal = list(pool.imap( open_tree, f_signal))

MaybeEncodingError: Error sending result: '<utils.classUtils.Tree.Tree object at 0x7f06283373a0>'. Reason: 'TypeError("cannot pickle '_global_parameters' object")'

In [28]:
def get_jet_p4(tree):
    p4 = build_p4(tree, prefix='jet')
    return p4.pt

list(pool.imap(get_jet_p4, signal))

[<Array [[202, 170, 115, ... 70.7, 51.9, 23.3]] type='133604 * var * float32'>,
 <Array [[422, 389, 86.7, ... 37.4, 31.8, 23.6]] type='85711 * var * float32'>]

In [30]:
def load_feynnet(tree, model=eightb.models.feynnet_bkg_33sig):
    eightb.load_feynnet_assignment(tree, model.storage)
    return tree

list(pool.imap(load_feynnet, signal))

Process ForkPoolWorker-5:
Process ForkPoolWorker-6:
Traceback (most recent call last):
  File "/uscms_data/d3/ekoenig/mambaforge/envs/py-env/lib/python3.9/multiprocessing/process.py", line 315, in _bootstrap
    self.run()
  File "/uscms_data/d3/ekoenig/mambaforge/envs/py-env/lib/python3.9/multiprocessing/process.py", line 108, in run
    self._target(*self._args, **self._kwargs)
  File "/uscms_data/d3/ekoenig/mambaforge/envs/py-env/lib/python3.9/multiprocessing/pool.py", line 114, in worker
    task = get()


KeyboardInterrupt: 

  File "/uscms_data/d3/ekoenig/mambaforge/envs/py-env/lib/python3.9/multiprocessing/queues.py", line 365, in get
    res = self._reader.recv_bytes()
  File "/uscms_data/d3/ekoenig/mambaforge/envs/py-env/lib/python3.9/multiprocessing/connection.py", line 221, in recv_bytes
    buf = self._recv_bytes(maxlength)
  File "/uscms_data/d3/ekoenig/mambaforge/envs/py-env/lib/python3.9/multiprocessing/connection.py", line 419, in _recv_bytes
    buf = self._recv(4)
  File "/uscms_data/d3/ekoenig/mambaforge/envs/py-env/lib/python3.9/multiprocessing/connection.py", line 384, in _recv
    chunk = read(handle, remaining)
KeyboardInterrupt


Traceback (most recent call last):
  File "/uscms_data/d3/ekoenig/mambaforge/envs/py-env/lib/python3.9/multiprocessing/process.py", line 315, in _bootstrap
    self.run()
  File "/uscms_data/d3/ekoenig/mambaforge/envs/py-env/lib/python3.9/multiprocessing/process.py", line 108, in run
    self._target(*self._args, **self._kwargs)
  File "/uscms_data/d3/ekoenig/mambaforge/envs/py-env/lib/python3.9/multiprocessing/pool.py", line 114, in worker
    task = get()
  File "/uscms_data/d3/ekoenig/mambaforge/envs/py-env/lib/python3.9/multiprocessing/queues.py", line 364, in get
    with self._rlock:
  File "/uscms_data/d3/ekoenig/mambaforge/envs/py-env/lib/python3.9/multiprocessing/synchronize.py", line 95, in __enter__
    return self._semlock.__enter__()
KeyboardInterrupt
